**TODO**

* **TODO** Rerun eps = 0.9, mod = 10 000

In [23]:
%matplotlib qt

In [24]:
import os
from pydgilib_extra import DGILibPlot
from experiment.averages import Averages
import pandas as pd
import matplotlib.pyplot as plt
import operator
from IPython.display import display, HTML

def wait_for_plot(fig):
    if os.name != "nt":
        while plt.fignum_exists(fig.number):
            plt.pause(0.000001)

class ExperimentResult(object):
    
    def __init__(self,folder="baseline_2887iter_1", output_dir = "C:\\Users\\Dragos\\Dropbox\\RISE\\Git\\PythonCSV\\output"):
        
        self.folder = folder
        self.output_dir = output_dir
        
        self.averages_csv = os.path.join(output_dir, folder, folder + "_averages.csv")
        self.table_stats_txt = os.path.join(output_dir, folder, folder + "_table_stats.txt")
        self.output_csv = os.path.join(output_dir, folder, folder + "_output.csv")
        
        self.column_names = ['Iteration','Roll', 'Pitch', 'Gyro X Angle', 'Gyro Y Angle', 'Comp Angle X', 'Comp Angle Y', 'Kalman Angle X', 'Kalman Angle Y']
        
        self.output_axes = None
        self.output_fig = None
        self.output_df = pd.read_csv(self.output_csv, header = None, names = self.column_names)
        self.output_colors = ['red', 'blue', 'black', 'brown']
        self.output_plotted_axes = 0
        
        self.legend = ""
        if "hash" in folder:
            self.legend = " ".join(folder.split("_")[0:6]).title()
        else:
            self.legend = folder.split("_")[0].title()
        
        self.iterations = min(len(self.output_df['Iteration']), int(folder.split("iter")[0].split("_")[-1]))
        
        self.avgobj = Averages()
        
        self.error = {
            'Roll': 0,
            'Pitch': 0,
            'Gyro X Angle': 0,
            'Gyro Y Angle': 0,
            'Comp Angle X': 0,
            'Comp Angle Y': 0,
            'Kalman Angle X': 0,
            'Kalman Angle Y': 0,
            'Average': 0            
        }
        
    def plot_output(self, color='red', reference = []):
        self.output_fig, self.output_axes = plt.subplots(nrows=2, ncols=4)

        fig, axes = self.output_fig, self.output_axes
        dfs = [self.output_df]
        column_names = self.column_names
        legends = [self.legend]
        
        for exp in reference:
            legends.append(exp.legend)
            dfs.append(exp.output_df)
        
        for i in range(1,len(column_names)):
            row = int((i-1)/4)
            column = int((i-1)%4)
            
            color_idx = 0
            for df in dfs:
                df.plot(kind='line', x='Iteration', y=column_names[i], ax=axes[row, column], color=self.output_colors[color_idx])
                color_idx += 1
            
            axes[row, column].legend(legends)
            axes[row, column].set_title(column_names[i])

        self.output_axes = axes
        self.output_fig = fig
        
        wait_for_plot(fig)
        
    def plot_power_curves(self, reference=None):
        run_name_1 = self.folder
        
        if reference is None:
            reference = ExperimentResult()
        
        run_name_2 = reference.folder
        
        filename_1 = os.path.join(self.output_dir, run_name_1, run_name_1 + "_averages.csv")
        filename_2 = os.path.join(reference.output_dir, run_name_2, run_name_2 + "_averages.csv")

        column_names = ['Pin','Iteration', 'From', 'To', 'Charge']
        df1 = pd.read_csv(filename_1, header = None, names=column_names)
        df2 = pd.read_csv(filename_2, header = None, names=column_names)

        iterations = int(run_name_1.split("iter")[0].split("_")[-1])

        df1 = df1[1:iterations]
        df2 = df2[1:iterations]

        df1["Charge"] = df1["Charge"].astype(float) #pd.to_numeric(df1["Charge"])
        df2["Charge"] = df2["Charge"].astype(float)

        fig, axes = plt.subplots(nrows=1, ncols=1)
        
        df1.plot(kind='line', x='Iteration', y='Charge', ax=axes, color='blue')
        df2.plot(kind='line', x='Iteration', y='Charge', ax=axes, color='black')
        
        axes.legend([self.legend, reference.legend])
        
        wait_for_plot(fig)
        
    def calculate_deviation(self, reference=None):
        if self.error["Average"] > 0.0:
            return
        
        if reference is None:
            reference = ExperimentResult()
        
        df = self.output_df
        df2 = reference.output_df
        
        iterations = min(self.iterations, len(df2['Iteration']))
        
        for column in self.column_names:
            if column == 'Iteration': continue
            for i in range(iterations):
                if df2[column][i] != 0.0: 
                    val = abs(df[column][i] - df2[column][i] / df2[column][i])
                else: 
                    val = 0.0
                self.error[column] += val
                #print("Adding {0} for column {1} and i {2}".format(val, column, i))
                
            self.error["Average"] += self.error[column]
            self.error[column] /= iterations
            
        self.error["Average"] /= (iterations * (len(self.column_names) - 1))
        
    def print_deviation(self,verbose=1):
        if verbose == 1:
            print("Error: {0}%".format(self.error["Average"]))
        elif verbose >= 2:
            for column in self.column_names:
                if 'Iteration' == column: continue
                print("{0}: {1:0.4f}%".format(column, self.error[column]))
            print("Average error: {0:0.4f}%".format(self.error["Average"]))
    
    def calculate_averages(self):
        self.avgobj.read_from_csv(self.averages_csv)
        self.avgobj.calculate_averages_for_pin(1)
    
    def show_averages(self, verbose=1):
        
        print_per_iteration=False
        print_total_average=False
        print_benchmark_time=False
        print_energy_and_current=False
        
        if verbose >= 1:
            print_per_iteration = True
        if verbose >= 2:
            print_energy_and_current = True
            print_total_average = True
        if verbose >= 3:
            print_benchmark_time = True
        
        self.avgobj.print_averages_for_pin(1, print_per_iteration = print_per_iteration, 
                                          print_energy_and_current = print_energy_and_current,
                                          print_total_average = print_total_average,
                                          print_benchmark_time = print_benchmark_time)
    
    def get_averages(self, pin = 1):
        return {
            'Average per iteration uC': self.avgobj.total_average[pin] * 1000 * 1000 / self.avgobj.total_iterations[pin],
            'Average per iteration duration': self.avgobj.total_duration[pin] / self.avgobj.total_iterations[pin],
            'Total mC': self.avgobj.total_average[pin] * 1000,
            'Total duration': self.avgobj.total_duration[1],
            'True iterations': self.avgobj.total_iterations[pin]
        }
    
    def get_error(self):
        return {
            'Average': self.error["Average"], 
            'List': list(self.error.values())[:-1],
            'Maximum': max(list(self.error.values())[:-1]),
            'Maximum name': max(self.error.items(), key=operator.itemgetter(1))[0]
        }
    
    def get_hash_stats(self, verbose=0):
        epsilon = 0
        epsilon_mod = 0
        mod_precision = 0
        hash_size = 0
        utilizied = 0
        found = 0
        stored = 0

        for line in open(self.table_stats_txt, "r"):
            if ("Epsilon:" in line):
                epsilon = float(line.split(" ")[-1])
            elif ("Epsilon mod" in line):
                epsilon_mod = int(line.split(" ")[-1])
            elif ("Mod precision" in line):
                mod_precision = int(line.split(" ")[-1])
            elif ("Hash size" in line):
                hash_size = int(line.split(" ")[-1])
            elif ("HashTable population" in line):
                utilized = int(line.split(" ")[-3])
            elif ("found" in line):
                found = int(line.split(" ")[-1])
            elif ("stored" in line):
                stored = int(line.split(" ")[-1])
            if verbose>0:print(line, end='')

        return {
            'Epsilon': epsilon,
            'Epsilon Mod': epsilon_mod, 
            'Mod precision': mod_precision,
            'Hash size': hash_size,
            'Utilized': utilized,
            'Found': found,
            'Stored': stored
        }
        
class ResultsTable(object):
    
    def __init__(self):
        
        self.column_names = ["Type", "Epsilon", "Mod precision", 
                             "mC/iter", "sec/iter", "Total mC", 
                             "Total sec",
                             "Accuracy average", "Accuracy worst", "Accuracy worst name",
                             "Hits from hash", "Writes to hash", "Iterations"]
        self.df = pd.DataFrame(columns=self.column_names)
        
    def add_to_table(self, exp):
        averages = exp.get_averages()
        errors = exp.get_error()
        
        if "baseline" in exp.folder:
            row = ["Baseline", 0, 0, 
                   averages['Average per iteration uC'], averages['Average per iteration duration'], averages['Total mC'],
                   averages['Total duration'], 
                   errors['Average'], errors['Maximum'], errors['Maximum name'],
                   0, 0, 0]
            assert(len(row) == len(self.column_names))
        else:
            hash_stats = exp.get_hash_stats()
            row = ["Hash", hash_stats['Epsilon'], hash_stats['Mod precision'], averages['Average per iteration uC'],
                   averages['Average per iteration duration'], averages['Total mC'], averages['Total duration'],
                   errors['Average'], errors['Maximum'], errors['Maximum name'],
                   hash_stats['Found'], hash_stats['Stored'], averages['True iterations']]
            assert(len(row) == len(self.column_names))
        #print(len(self.column_names), len(row))
        self.df.loc[len(self.df)] = row
        
        self.df["Mod precision"] = self.df["Mod precision"].astype(int)
        self.df["Hits from hash"] = self.df["Hits from hash"].astype(int)
        self.df["Writes to hash"] = self.df["Writes to hash"].astype(int)
        self.df["Iterations"] = self.df["Iterations"].astype(int)
        
    def print_table(self):
        display(self.df)

In [25]:
output_dir = "C:\\Users\\Dragos\\Dropbox\\RISE\\Git\\PythonCSV\\output"

if os.name == "nt":
    output_dir = "C:\\Users\\Dragos\\Dropbox\\RISE\\Git\\PythonCSV\\output"
else:
    output_dir = "/home/dragos/Dropbox/RISE/Git/PythonCSV/output/"

baseline = ExperimentResult(output_dir=output_dir)
hash120_eps05 = ExperimentResult(folder="hash_120_eps_0.5_mod_10000_2887iter_1", output_dir=output_dir)
hash120_eps06 = ExperimentResult(folder="hash_120_eps_0.6_mod_10000_2887iter_1", output_dir=output_dir)
hash120_eps07 = ExperimentResult(folder="hash_120_eps_0.7_mod_10000_2887iter_1", output_dir=output_dir)
hash120_eps08 = ExperimentResult(folder="hash_120_eps_0.8_mod_10000_2887iter_1", output_dir=output_dir)
hash120_eps085 = ExperimentResult(folder="hash_120_eps_0.85_mod_10000_2887iter_1", output_dir=output_dir)
hash120_eps09 = ExperimentResult(folder="hash_120_eps_0.9_mod_10000_2887iter_1", output_dir=output_dir)
hash120_eps094 = ExperimentResult(folder="hash_120_eps_0.94_mod_10000_2887iter_1", output_dir=output_dir)
hash120_eps095 = ExperimentResult(folder="hash_120_eps_0.95_mod_10000_2887iter_1", output_dir=output_dir)
hash120_eps1 = ExperimentResult(folder="hash_120_eps_1.0_mod_10000_2887iter_1", output_dir=output_dir)

FileNotFoundError: [Errno 2] File b'C:\\Users\\Dragos\\Dropbox\\RISE\\Git\\PythonCSV\\output\\baseline_2887iter_1\\baseline_2887iter_1_output.csv' does not exist: b'C:\\Users\\Dragos\\Dropbox\\RISE\\Git\\PythonCSV\\output\\baseline_2887iter_1\\baseline_2887iter_1_output.csv'

**Calculate everything**

In [19]:
# Mod precision: 10 000 for all
experiments = [baseline, hash120_eps05, hash120_eps06, hash120_eps07, hash120_eps08, hash120_eps085, hash120_eps09, hash120_eps094, hash120_eps095, hash120_eps1]

for e in experiments:
    e.calculate_deviation(reference=baseline)
    e.calculate_averages()

**Data in a table**

In [20]:
rt = ResultsTable()

for e in experiments:
    rt.add_to_table(e)


In [21]:
rt.print_table()

,Type,Epsilon,Mod precision,mC/iter,sec/iter,Total mC,Total sec,Accuracy average,Accuracy worst,Accuracy worst name,Hits from hash,Writes to hash,Iterations
0,Baseline,0,0,5.838719,0.011489,16.850542,33.158087,54.613159,102.337688,Gyro X Angle,0,0,0
1,Hash,0.5,10000,4.804306,0.009466,13.865226,27.317836,54.356917,102.440179,Gyro X Angle,710,2177,2886
2,Hash,0.6,10000,4.325328,0.008440,12.482897,24.356860,54.262571,102.362410,Gyro X Angle,971,1916,2886
3,Hash,0.7,10000,3.848934,0.007642,11.108024,22.053849,54.238345,102.311780,Gyro X Angle,1173,1714,2886
4,Hash,0.8,10000,3.470928,0.006902,10.017098,19.918327,54.086107,102.048708,Gyro X Angle,1358,1529,2886
5,Hash,0.85,10000,3.451650,0.006868,9.961463,19.819631,54.159042,102.305559,Gyro X Angle,1367,1520,2886
6,Hash,0.9,10000,3.229410,0.006441,9.316847,18.581567,53.800566,102.073252,Gyro X Angle,1475,1412,2885
7,Hash,0.94,10000,3.090601,0.006197,8.919476,17.883871,53.813233,102.153439,Gyro X Angle,1538,1349,2886
8,Hash,0.95,10000,3.024653,0.006072,8.729149,17.524155,53.833371,102.017890,Gyro X Angle,1569,1318,2886
9,Hash,1,10000,2.988586,0.005855,8.625060,16.896219,53.765784,101.926993,Gyro X Angle,1621,1266,2886


**Plots**

Output plots

In [32]:
hash120_eps1.plot_output(reference=[baseline])

In [ ]:
hash120_eps095.plot_output(reference=[baseline])

In [ ]:
hash120_eps094.plot_output(reference=[baseline])

In [ ]:
hash120_eps09.plot_output(reference=[baseline])

In [50]:
hash120_eps085.plot_output(reference=[baseline])

Power plots

In [33]:
hash120_eps1.plot_power_curves(reference=baseline)

In [ ]:
hash120_eps095.plot_power_curves(reference=baseline)

In [ ]:
hash120_eps094.plot_power_curves(reference=baseline)

In [ ]:
hash120_eps09.plot_power_curves(reference=baseline)

In [34]:
hash120_eps085.plot_power_curves(reference=baseline)

**Data in a table (testing)**

In [9]:
hash120_eps09.plot_output()

In [7]:
hash120_eps094.get_averages()

{'Average per iteration charge': 0.0030906013916907806,
 'Average per iteration duration': 0.006196767475167963,
 'Total charge': 8.919475616419593,
 'Total duration': 17.88387093333474}

In [22]:
hash120_eps094.get_error()

{'Average': 54.61315937110321,
 'List': [96.52031448077588,
  12.450947742639443,
  102.33768848908889,
  14.272661107377902,
  96.78228715552473,
  5.60654353827501,
  96.48779968721857,
  12.447032767925206],
 'Maximum': 102.33768848908889,
 'Maximum name': 'Gyro X Angle'}

In [9]:
hash120_eps094.get_hash_stats()

{'Epsilon': 0.94,
 'Epsilon Mod': 9400,
 'Mod precision': 10000,
 'Hash size': 120,
 'Utilized': 4,
 'Found': 1538,
 'Stored': 1349}

In [ ]:
baseline.calculate_deviation(reference=baseline)

In [ ]:
baseline.get_deviation()

**Data in text format**

In [80]:
print("Baseline")
print("---")
baseline.print_output_error(verbose=2)
print("---")
baseline.show_averages(verbose=2)

Baseline
---
Roll: 96.5203%
Pitch: 12.4509%
Gyro X Angle: 102.3377%
Gyro Y Angle: 14.2727%
Comp Angle X: 96.7823%
Comp Angle Y: 5.6065%
Kalman Angle X: 96.4878%
Kalman Angle Y: 12.4470%
Average error: 54.6132%
---
Average charge per iteration: 5.83871872 uC
Average energy per iteration: 29.193594 uJ
Average time per iteration: 0.011489 ms

Total iterations: 2886
Total average current: 0.508188 mA
Total charge: 16.850542 mC
Total energy: 84.252711 mJ
Total time: 33.158087 s



In [71]:
print("Hash size: 120, Epsilon: 1, Mod precision: 10000")
print("---")
hash120_eps1.print_output_error(verbose=2)
print("---")
hash120_eps1.show_averages(verbose=2)

Hash size: 120, Epsilon: 1, Mod precision: 10000
---
Roll: 94.9683%
Pitch: 11.8805%
Gyro X Angle: 101.9270%
Gyro Y Angle: 12.9564%
Comp Angle X: 95.5064%
Comp Angle Y: 6.1134%
Kalman Angle X: 94.9494%
Kalman Angle Y: 11.8250%
Average error: 53.7658%
---
Average charge per iteration: 2.988586131 uC
Average energy per iteration: 14.942931 uJ
Average time per iteration: 0.005855 ms

Total iterations: 2886
Total average current: 0.510473 mA
Total charge: 8.62506 mC
Total energy: 43.125298 mJ
Total time: 16.896219 s



In [72]:
print("Hash size: 120, Epsilon: 0.95, Mod precision: 10000")
print("---")
hash120_eps095.print_output_error(verbose=2)
print("---")
hash120_eps095.show_averages(verbose=2)

Hash size: 120, Epsilon: 0.95, Mod precision: 10000
---
Roll: 95.1148%
Pitch: 11.9365%
Gyro X Angle: 102.0179%
Gyro Y Angle: 13.1073%
Comp Angle X: 95.4448%
Comp Angle Y: 6.0554%
Kalman Angle X: 95.0996%
Kalman Angle Y: 11.8907%
Average error: 53.8334%
---
Average charge per iteration: 3.024653009 uC
Average energy per iteration: 15.123265 uJ
Average time per iteration: 0.006072 ms

Total iterations: 2886
Total average current: 0.498121 mA
Total charge: 8.729149 mC
Total energy: 43.645743 mJ
Total time: 17.524155 s



In [73]:
print("Hash size: 120, Epsilon: 0.94, Mod precision: 10000")
print("---")
hash120_eps094.print_output_error(verbose=2)
print("---")
hash120_eps094.show_averages(verbose=2)

Hash size: 120, Epsilon: 0.94, Mod precision: 10000
---
Roll: 95.1172%
Pitch: 12.0202%
Gyro X Angle: 102.1534%
Gyro Y Angle: 12.6506%
Comp Angle X: 95.5954%
Comp Angle Y: 5.8923%
Kalman Angle X: 95.1014%
Kalman Angle Y: 11.9752%
Average error: 53.8132%
---
Average charge per iteration: 3.090601392 uC
Average energy per iteration: 15.453007 uJ
Average time per iteration: 0.006197 ms

Total iterations: 2886
Total average current: 0.498744 mA
Total charge: 8.919476 mC
Total energy: 44.597378 mJ
Total time: 17.883871 s



**Output Plots**